In [3]:
import os
import csv
import numpy as np
import requests
from shapely.geometry import Point, Polygon

# 建立存放 CSV 的資料夾
output_dir = "1_grid_points"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 定義一個粗略近似台灣主島的多邊形（座標格式：(lon, lat)）
taiwan_polygon = Polygon([
    (120.1, 21.9),  # 近南端（屏東附近）
    (121.0, 21.9),  # 近東南
    (122.0, 22.5),  # 東部海岸往北延伸
    (122.0, 23.5),  # 東部中北部
    (121.5, 24.5),  # 北部偏東
    (120.5, 25.1),  # 北部偏西
    (119.5, 24.8),  # 西部北端
    (119.0, 23.5),  # 西部中南部
    (119.0, 22.5),  # 西部南端
    (120.1, 21.9)   # 回到起點
])

# 設定網格範圍與間距
lon_min, lon_max = 119.0, 122.5
lat_min, lat_max = 21.5, 25.5
step = 0.25

# 產生網格點，並利用多邊形篩選台灣陸地上的點
points = []
lats = np.arange(lat_min, lat_max + step, step)
lons = np.arange(lon_min, lon_max + step, step)
for lat in lats:
    for lon in lons:
        point = Point(lon, lat)  # shapely 的 Point 順序為 (lon, lat)
        if taiwan_polygon.contains(point):
            # 儲存時注意 open-elevation API 需要的欄位名稱為 latitude 與 longitude
            points.append({"latitude": lat, "longitude": lon})

print("符合條件的點數:", len(points))

# 利用 open-elevation API 取得海拔資料
api_url = "https://api.open-elevation.com/api/v1/lookup"
batch_size = 100  # 一次查詢的點數，可根據需要調整
results = []

for i in range(0, len(points), batch_size):
    batch = points[i:i + batch_size]
    payload = {"locations": batch}
    try:
        response = requests.post(api_url, json=payload)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"批次從索引 {i} 發生錯誤: {e}")
        continue

    data = response.json()
    if "results" in data:
        results.extend(data["results"])
    else:
        print(f"批次從索引 {i} 回傳資料格式錯誤: {data}")

# 將結果寫入 CSV
csv_path = os.path.join(output_dir, "taiwan_grid.csv")
with open(csv_path, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["lon", "lat", "elevation_m"])  # CSV 標題
    # API 回傳的每個項目含有 latitude, longitude, elevation
    for entry in results:
        writer.writerow([entry["longitude"], entry["latitude"], entry["elevation"]])

print("CSV 檔案已儲存於:", csv_path)


符合條件的點數: 116
CSV 檔案已儲存於: 1_grid_points\taiwan_grid.csv
